In [13]:
import requests
from abc import ABC, abstractmethod
from typing import List, Optional
import importlib
import backend.adapters.repository as repository
import backend.domain.model as model
import backend.service_layer.service as service
import backend.service_layer.uow as uow
from sec_api import XbrlApi
import os
from backend.domain.model import Company, Filing

In [14]:
uow_instance = uow.UnitOfWork()
company = service.get_company_by_ticker('GES', uow_instance)


In [15]:
company.filings = company.get_filings_by_type('10-Q')

In [16]:
filing = company.filings[0]

In [17]:
filing.data = uow_instance.sec_filings.get_filing_data(
        filing.cik,
        filing.accession_number,
        filing.primary_document
    )

In [18]:
filing._data

{'CoverPage': {'DocumentType': '10-Q',
  'DocumentQuarterlyReport': 'true',
  'DocumentPeriodEndDate': '2024-11-02',
  'DocumentTransitionReport': 'false',
  'EntityFileNumber': '1-11893',
  'EntityIncorporationStateCountryCode': 'DE',
  'EntityTaxIdentificationNumber': '95-3679695',
  'EntityAddressAddressLine1': 'Strada Regina 44',
  'EntityAddressCityOrTown': 'Bioggio',
  'EntityAddressCountry': 'CH',
  'EntityAddressPostalZipCode': 'CH-6934',
  'CityAreaCode': '(213)',
  'LocalPhoneNumber': '765-3100',
  'Security12bTitle': 'Common Stock, par value $0.01 per share',
  'TradingSymbol': 'GES',
  'SecurityExchangeName': 'NYSE',
  'EntityCurrentReportingStatus': 'Yes',
  'EntityInteractiveDataCurrent': 'Yes',
  'EntityFilerCategory': 'Accelerated Filer',
  'EntitySmallBusiness': 'false',
  'EntityEmergingGrowthCompany': 'false',
  'EntityShellCompany': 'false',
  'EntityCommonStockSharesOutstanding': {'decimals': 'INF',
   'unitRef': 'shares',
   'period': {'instant': '2024-12-02'},
  

In [20]:
combined = service.get_consolidated_income_statements('GES', uow_instance, '10-Q')

Years covered in last filing (2025-04-11):
Index(['2022-01-30:2023-01-28', '2023-01-29:2024-02-03',
       '2024-02-04:2025-02-01'],
      dtype='object', name='date_range')
Index(['2022-01-30:2023-01-28', '2023-01-29:2024-02-03',
       '2024-02-04:2025-02-01'],
      dtype='object', name='date_range')
Index(['2022-01-30:2023-01-28', '2023-01-29:2024-02-03',
       '2024-02-04:2025-02-01'],
      dtype='object', name='date_range')
Years covered in last filing (2024-12-06):
Index(['2023-07-30:2023-10-28', '2024-08-04:2024-11-02'], dtype='object', name='date_range')
Index(['2023-07-30:2023-10-28', '2024-08-04:2024-11-02'], dtype='object', name='date_range')
Index(['2023-07-30:2023-10-28', '2024-08-04:2024-11-02'], dtype='object', name='date_range')
Error getting filing data for 0000912463 0001047469-10-002994 a2197641z10-k.htm: API error: 404 - {"status":404,"error":"The filer did not attach XBRL data to the filing. A conversion is not possible."}
Error getting filing data for 000091246

Traceback (most recent call last):
  File "c:\Users\Site Oracle\reduceComplexity\backend\adapters\repository.py", line 192, in map_dataframes
    WeightedAverageNumberOfSharesOutstanding, and WeightedAverageNumberOfSharesDiluted.
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\google\generativeai\generative_models.py", line 331, in generate_content
    response = self._client.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\google\ai\generativelanguage_v1beta\services\generative_service\client.py", line 835, in generate_content
    response = rpc(
               ^^^^
  File "c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\google\api_core\gapic_v1\method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Sit

Retry attempt 1 for mapping. Problematic indices: []
Sending prompt to LLM with 23 rows from DF1 and 22 rows from DF2
Unexpected error in mapping (attempt 2/3): 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 35
}
]
Retry attempt 2 for mapping. Problematic indices: []
Sending prompt to LLM with 23 rows from DF1 and 22 rows from DF2


Traceback (most recent call last):
  File "c:\Users\Site Oracle\reduceComplexity\backend\adapters\repository.py", line 192, in map_dataframes
    WeightedAverageNumberOfSharesOutstanding, and WeightedAverageNumberOfSharesDiluted.
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\google\generativeai\generative_models.py", line 331, in generate_content
    response = self._client.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\google\ai\generativelanguage_v1beta\services\generative_service\client.py", line 835, in generate_content
    response = rpc(
               ^^^^
  File "c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\google\api_core\gapic_v1\method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Sit

Unexpected error in mapping (attempt 3/3): 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 35
}
]
Maximum retries reached. Returning empty mapping.

=== DEBUG: DATAFRAME MAPPING ===
DF1 index types: ['AssetImpairmentCharges', 'CostOfGoodsAndServicesSold', 'EarningsPerShareBasic', 'EarningsPerShareDiluted', 'GainLossOnTerminationOfLease', 'GainsLossesOnExtinguishmentOfDebt', 'GrossProfit', 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest', 'IncomeLossFromEquityMethodInvestments', 'IncomeTaxExpenseBenefit', 'InterestExpenseNonoperating', 'InvestmentIncomeInterest', 'NetIncomeLoss', 'NetIncomeLossAttributableToNoncontrollingInterest', 'NonoperatingIncomeEx

...

Traceback (most recent call last):
  File "c:\Users\Site Oracle\reduceComplexity\backend\adapters\repository.py", line 192, in map_dataframes
    WeightedAverageNumberOfSharesOutstanding, and WeightedAverageNumberOfSharesDiluted.
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\google\generativeai\generative_models.py", line 331, in generate_content
    response = self._client.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\google\ai\generativelanguage_v1beta\services\generative_service\client.py", line 835, in generate_content
    response = rpc(
               ^^^^
  File "c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\google\api_core\gapic_v1\method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Sit

Unexpected error in mapping (attempt 3/3): 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 27
}
]
Maximum retries reached. Returning empty mapping.
Sending prompt to LLM to make 23 index names readable
Unexpected error in make_index_readable (attempt 1/3): 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 27
}
]
Sending prompt to LLM to make 23 index names readable


Traceback (most recent call last):
  File "c:\Users\Site Oracle\reduceComplexity\backend\adapters\repository.py", line 192, in map_dataframes
    WeightedAverageNumberOfSharesOutstanding, and WeightedAverageNumberOfSharesDiluted.
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\google\generativeai\generative_models.py", line 331, in generate_content
    response = self._client.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\google\ai\generativelanguage_v1beta\services\generative_service\client.py", line 835, in generate_content
    response = rpc(
               ^^^^
  File "c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\google\api_core\gapic_v1\method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Sit

Unexpected error in make_index_readable (attempt 2/3): 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 27
}
]
Sending prompt to LLM to make 23 index names readable
Unexpected error in make_index_readable (attempt 3/3): 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 27
}
]
Maximum retries reached. Returning unchanged names.


Traceback (most recent call last):
  File "c:\Users\Site Oracle\reduceComplexity\backend\adapters\repository.py", line 282, in make_index_readable
    Original names: {list(index_names)}
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\google\generativeai\generative_models.py", line 331, in generate_content
    response = self._client.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\google\ai\generativelanguage_v1beta\services\generative_service\client.py", line 835, in generate_content
    response = rpc(
               ^^^^
  File "c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\google\api_core\gapic_v1\method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\google\api_core\retry\retry_unary.py", li

In [38]:
sorted_columns = sorted(combined.df.columns, key=lambda x: x.split(':')[0])
print(sorted_columns)

['2009-05-03:2009-08-01', '2009-08-02:2009-10-31', '2010-01-31:2011-01-29', '2010-05-02:2010-07-31', '2010-08-01:2010-10-30', '2011-01-30:2012-01-28', '2011-01-30:2011-04-30', '2011-05-01:2011-07-30', '2011-07-31:2011-10-29', '2012-01-29:2013-02-02', '2012-01-29:2012-04-28', '2012-04-29:2012-07-28', '2012-07-29:2012-10-27', '2013-02-03:2014-02-01', '2013-02-03:2013-05-04', '2013-05-05:2013-08-03', '2013-08-04:2013-11-02', '2014-02-02:2015-01-31', '2014-02-02:2014-05-03', '2014-05-04:2014-08-02', '2014-08-03:2014-11-01', '2015-02-01:2016-01-30', '2015-02-01:2015-05-02', '2015-05-03:2015-08-01', '2015-08-02:2015-10-31', '2016-01-31:2017-01-28', '2016-01-31:2016-04-30', '2016-05-01:2016-07-30', '2016-07-31:2016-10-29', '2017-01-29:2018-02-03', '2017-01-29:2017-04-29', '2017-04-30:2017-07-29', '2017-07-30:2017-10-28', '2018-02-04:2019-02-02', '2018-02-04:2018-05-05', '2018-05-06:2018-08-04', '2018-08-05:2018-11-03', '2019-02-03:2020-02-01', '2019-02-03:2019-05-04', '2019-05-05:2019-08-03',

In [19]:
# Get data for first 4 filings
for filing in company.filings[:1]:
    filing.data = uow_instance.sec_filings.get_filing_data(
        filing.cik,
        filing.accession_number,
        filing.primary_document
    )
    filing.filing_url = uow_instance.sec_filings.get_filing_url(
        filing.cik,
        filing.accession_number,
        filing.primary_document
    )
    print(f'Loaded data for filing date {filing.filing_date}, form {filing.form}')


Loaded data for filing date 2024-12-06, form 10-Q


In [ ]:
res = company.select_filings_with_processing_pattern(company.filings,'10-Q')

Years covered in last filing (2024-12-06):
Index(['2023-07-30:2023-10-28', '2024-08-04:2024-11-02'], dtype='object', name='date_range')
Index(['2023-07-30:2023-10-28', '2024-08-04:2024-11-02'], dtype='object', name='date_range')
Index(['2023-07-30:2023-10-28', '2024-08-04:2024-11-02'], dtype='object', name='date_range')


In [21]:
for filing in res:
    print(f'Loaded data for filing date {filing.filing_date}, form {filing.form}')

Loaded data for filing date 2024-12-06, form 10-Q
Loaded data for filing date 2024-09-06, form 10-Q
Loaded data for filing date 2024-06-07, form 10-Q
Loaded data for filing date 2022-12-02, form 10-Q
Loaded data for filing date 2022-09-02, form 10-Q
Loaded data for filing date 2022-06-02, form 10-Q
Loaded data for filing date 2020-12-04, form 10-Q
Loaded data for filing date 2020-09-04, form 10-Q
Loaded data for filing date 2020-06-11, form 10-Q
Loaded data for filing date 2018-12-07, form 10-Q
Loaded data for filing date 2018-09-06, form 10-Q
Loaded data for filing date 2018-06-07, form 10-Q
Loaded data for filing date 2016-12-06, form 10-Q
Loaded data for filing date 2016-09-01, form 10-Q
Loaded data for filing date 2016-06-02, form 10-Q
Loaded data for filing date 2014-12-09, form 10-Q
Loaded data for filing date 2014-09-05, form 10-Q
Loaded data for filing date 2014-06-06, form 10-Q
Loaded data for filing date 2012-12-05, form 10-Q
Loaded data for filing date 2012-09-05, form 10-Q
